In [11]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
import math
import time
from sklearn.metrics import mean_squared_error

In [1]:
df_train = pd.read_csv('data/DRdataTrain.csv')
df_test = pd.read_csv('data/DRdataTest.csv')

NameError: name 'pd' is not defined

In [13]:
df_test.shape

(22620, 15)

In [14]:
train_size = len(df_train)

train_X = df_train.iloc[:, :-1]
train_y = df_train.iloc[:,-1]
test_X = df_test.iloc[:, :-1]
test_y = df_test.iloc[:,-1]


In [17]:
train_X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,54,0.5,80,Benign,9,0.425820,0.052035,0.044126,0.028046,0.44997,1.00000,0.79423,0.59194,0.15127
1,325,6.0,80,Benign,15,0.000000,0.000000,0.000000,0.000000,1.00000,1.00000,1.00000,1.00000,1.00000
2,7,6.0,60,Benign,1,0.000000,0.000000,0.000000,0.000000,1.00000,0.90047,0.71260,0.39687,0.11078
3,4,5.0,20,Low,12,0.000000,0.000000,0.000000,0.000000,1.00000,0.90173,0.71260,0.39687,0.11078
4,66,2.5,60,Severe,1,0.000000,0.000000,0.000000,0.010694,0.98931,1.00000,0.98567,0.94057,0.41877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90475,4,2.5,80,Severe,3,0.000000,0.000000,0.000000,0.010694,0.98931,0.90179,0.71217,0.39687,0.11078
90476,84,5.0,40,Severe,18,0.000000,0.000000,0.000000,0.000000,1.00000,1.00000,0.86337,0.71133,0.20401
90477,86,1.0,40,moderate,3,0.092197,0.065878,0.058841,0.038933,0.74415,1.00000,0.96782,0.90208,0.35678
90478,92,0.0,80,Low,9,0.653970,0.000000,0.000000,0.000000,0.35019,1.00000,0.92496,0.81562,0.27088


In [16]:
scaler = MaxAbsScaler()
scaler.fit(train_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)

ValueError: could not convert string to float: 'Benign'

In [32]:
# poly = PolynomialFeatures(2)
# train_X = poly.fit_transform(train_X)
# test_X = poly.fit_transform(test_X)

In [9]:
from sklearn.base import BaseEstimator, ClassifierMixin


class GRNN(BaseEstimator, ClassifierMixin):
    def __init__(self, name = "GRNN", sigma = 0.1):
        self.name = name
        self.sigma = 2 * np.power(sigma, 2)
        
    def predict(self, instance_X, train_X, train_y):
        gausian_distances = np.exp(-np.power(np.sqrt((np.square(train_X-instance_X).sum(axis=1))),2)\
                                   / self.sigma)
        gausian_distances_sum = gausian_distances.sum()
        if gausian_distances_sum < math.pow(10, -7): gausian_distances_sum = math.pow(10, -7)
        result = np.multiply(gausian_distances, train_y).sum() / gausian_distances_sum
        return result

In [40]:
#np.savetxt('C:/Users/User/Desktop/gtm/GTM_PR.csv', pred_y, delimiter=',', fmt='%1.5f')

In [41]:

from sklearn.metrics import mean_absolute_percentage_error, max_error, median_absolute_error, r2_score, \
    mean_absolute_error
from sklearn.metrics import mean_squared_error

for s in np.arange(0.1, 10, 0.1):
    grnn = GRNN(sigma=s)
    start_time = time.time()
    predictions = np.apply_along_axis(lambda i: grnn.predict(i, train_X, train_y), axis=1, arr=test_X)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("Sigma: " + str(s))
    print('Testing errors:')
    print("MAPE: " + str(mean_absolute_percentage_error(test_y, predictions)))
    print("RMSE: " + str(root_mean_squared_error(test_y, predictions)))
    print()
    
    
    print("MAPE: " + str(mean_absolute_percentage_error(test_y, test_y_final_predictions)))
    print("RMSE: " + str(root_mean_squared_error(test_y, test_y_final_predictions)))
    print("MAE: " + str(mean_absolute_error(test_y, test_y_final_predictions)))
    print("R2: " + str(r2_score(test_y, test_y_final_predictions)))
    print("MSE: " + str(mean_squared_error(test_y, test_y_final_predictions)))
    print("ME: " + str(max_error(test_y, test_y_final_predictions)))
    print("MedAE: " + str(median_absolute_error(test_y, test_y_final_predictions)))

--- 5194.133347272873 seconds ---
Sigma: 0.1
Testing errors:
MAPE: 2.1324410152935016
RMSE: 2.159356887182757


In [42]:
# np.savetxt('C:/Users/User/Desktop/gtm/GRNN.csv', pred_y, delimiter=',', fmt='%1.5f')

In [43]:
from sklearn.svm import SVR
svr = SVR(kernel='rbf', gamma='auto', coef0=0.0, epsilon=0.001, max_iter=-1)
start_time = time.time()
svr.fit(train_X, train_y)
print("--- %s seconds ---" % (time.time() - start_time))
train_pred_y = svr.predict(train_X)
pred_y = svr.predict(test_X)

print('Training erros:')
print("MAPE: " + str(mean_absolute_percentage_error(train_y, train_pred_y)))
print("RMSE: " + str(root_mean_squared_error(train_y, train_pred_y)))
print()
print('Testing errors:')
print("MAPE: " + str(mean_absolute_percentage_error(test_y, pred_y)))
print("RMSE: " + str(root_mean_squared_error(test_y, pred_y)))

C:\Users\ivani\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


--- 6.470304727554321 seconds ---
Training erros:
MAPE: 11.258999530561468
RMSE: 9.567016806713871

Testing errors:
MAPE: 11.25495015795211
RMSE: 9.578003365045234


In [44]:
np.savetxt('C:/Users/User/Desktop/gtm/SVM.csv', pred_y, delimiter=',', fmt='%1.5f')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/User/Desktop/gtm/SVM.csv'

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor(loss = 'squared_loss', alpha=0.0001)
start_time = time.time()
sgd.fit(train_X, train_y)
print("--- %s seconds ---" % (time.time() - start_time))
train_pred_y = sgd.predict(train_X)
pred_y = sgd.predict(test_X)

print('Training erros:')
print("MAPE: " + str(mean_absolute_percentage_error(train_y, train_pred_y)))
print("RMSE: " + str(root_mean_squared_error(train_y, train_pred_y)))
print()
print('Testing errors:')
print("MAPE: " + str(mean_absolute_percentage_error(test_y, pred_y)))
print("RMSE: " + str(root_mean_squared_error(test_y, pred_y)))

In [ ]:
np.savetxt('C:/Users/User/Desktop/gtm/LR+Grad+PI.csv', pred_y, delimiter=',', fmt='%1.5f')

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

adaboost = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300)
start_time = time.time()
adaboost.fit(train_X, train_y)
print("--- %s seconds ---" % (time.time() - start_time))
train_pred_y = adaboost.predict(train_X)
pred_y = adaboost.predict(test_X)

print('Training erros:')
print("MAPE: " + str(mean_absolute_percentage_error(train_y, train_pred_y)))
print("RMSE: " + str(root_mean_squared_error(train_y, train_pred_y)))
print()
print('Testing errors:')
print("MAPE: " + str(mean_absolute_percentage_error(test_y, pred_y)))
print("RMSE: " + str(root_mean_squared_error(test_y, pred_y)))

In [ ]:
np.savetxt('C:/Users/User/Desktop/gtm/AdaBoost.csv', pred_y, delimiter=',', fmt='%1.5f')

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=5, random_state=0)
start_time = time.time()
regr.fit(train_X, train_y)
print("--- %s seconds ---" % (time.time() - start_time))
train_pred_y = regr.predict(train_X)
pred_y = regr.predict(test_X)

print('Training erros:')
print("MAPE: " + str(mean_absolute_percentage_error(train_y, train_pred_y)))
print("RMSE: " + str(root_mean_squared_error(train_y, train_pred_y)))
print()
print('Testing errors:')
print("MAPE: " + str(mean_absolute_percentage_error(test_y, pred_y)))
print("RMSE: " + str(root_mean_squared_error(test_y, pred_y)))

In [ ]:
np.savetxt('C:/Users/User/Desktop/gtm/RandomForest.csv', pred_y, delimiter=',', fmt='%1.5f')

In [ ]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(100, 40, 20), activation='relu',\
             solver='adam', alpha=0.0001, batch_size='auto', max_iter=200)
start_time = time.time()
mlp.fit(train_X, train_y)
print("--- %s seconds ---" % (time.time() - start_time))
train_pred_y = mlp.predict(train_X)
pred_y = mlp.predict(test_X)

print('Training erros:')
print("MAPE: " + str(mean_absolute_percentage_error(train_y, train_pred_y)))
print("RMSE: " + str(root_mean_squared_error(train_y, train_pred_y)))
print()
print('Testing errors:')
print("MAPE: " + str(mean_absolute_percentage_error(test_y, pred_y)))
print("RMSE: " + str(root_mean_squared_error(test_y, pred_y)))

In [ ]:
np.savetxt('C:/Users/User/Desktop/gtm/MPL.csv', pred_y, delimiter=',', fmt='%1.5f')

In [ ]:
+ ridge regression
+lasso regression